In [ ]:
# !pip install -U pymilvus
# !pip install -U dreamsim

In [8]:
import os
import numpy as np
from PIL import Image
import torch
from torchvision import models, transforms

# pymilvus のORM用モジュールとutilityをインポート
from pymilvus import connections, Collection, FieldSchema, CollectionSchema, DataType, utility

# --- Milvusへの接続確立 ---
connections.connect("default", uri="./milvus_demo.db")

# --- コレクションの作成 ---
collection_name = "image_embeddings"

# 既存のコレクションが存在する場合は削除
if utility.has_collection(collection_name):
    utility.drop_collection(collection_name)

# コレクションスキーマに "embedding" と "file_path" フィールドを定義
fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=2048),
    FieldSchema(name="file_path", dtype=DataType.VARCHAR, max_length=256)
]
schema = CollectionSchema(fields, description="画像埋め込みコレクション (Image Embedding Collection)")

# コレクションの作成
collection = Collection(name=collection_name, schema=schema)

# --- インデックス作成 ---
index_params = {
    "index_type": "IVF_FLAT",   # 倒立ファイルフラット
    "metric_type": "COSINE",      # コサイン類似度
    "params": {"nlist": 128}
}
collection.create_index(field_name="embedding", index_params=index_params)

# --- 画像特徴抽出器の設定 ---
# torchvisionのResNet50（レズネット50）を利用（全結合層をIdentityに変更して2048次元出力に）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet50(pretrained=True)
model.fc = torch.nn.Identity()  # fc層を除去
model.eval()
model.to(device)

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

def extract_feature(image_path):
    """画像パスから画像を読み込み、ResNet50で特徴ベクトルを抽出しL2正規化する関数"""
    img = Image.open(image_path).convert("RGB")
    img_tensor = preprocess(img).unsqueeze(0).to(device)
    with torch.no_grad():
        feature = model(img_tensor)
    feature_np = feature.cpu().numpy().flatten()
    norm = np.linalg.norm(feature_np)
    if norm > 0:
        feature_np = feature_np / norm
    return feature_np

# --- 画像データの登録 ---
# 画像が格納されたディレクトリ（例: "./images"）内のファイルを対象とします
image_dir = "./images"
image_files = [f for f in os.listdir(image_dir) if f.lower().endswith((".jpg", ".jpeg", ".png"))]

rows = []  # 行単位で挿入するリストを作成
for fname in image_files:
    path = os.path.join(image_dir, fname)
    feat = extract_feature(path)
    # 各行は辞書形式で "embedding" と "file_path" の両方を持たせる
    rows.append({
        "embedding": feat.tolist(),
        "file_path": path
    })

# 行単位でデータを挿入
collection.insert(rows)
collection.flush()  # 挿入完了を保証

# --- 画像検索クエリ ---
# 検索用画像（例: "./query.jpg"）から特徴を抽出し、類似画像を上位5件検索
query_image_path = "./query.jpg"  # 検索画像のパス
query_feature = extract_feature(query_image_path)

search_params = {
    "metric_type": "COSINE",      # コサイン類似度
    "params": {"nprobe": 10}
}
results = collection.search(
    data=[query_feature.tolist()],
    anns_field="embedding",
    param=search_params,
    limit=5,
    output_fields=["id", "file_path"]  # file_pathを出力に含める
)

# 検索結果の表示
print("検索結果:")
for i, hits in enumerate(results):
    print(f"クエリ {i} の結果:")
    for hit in hits:
        print(f"  ID: {hit.id}, 距離: {hit.distance}, ファイルパス: {hit.entity.get('file_path')}")


/home/kenta/miniforge3/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/kenta/miniforge3/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


検索結果:
クエリ 0 の結果:
  ID: 455852426864624279, 距離: 0.9999998807907104, ファイルパス: ./images/000000187976.jpg
  ID: 455852426864624249, 距離: 0.779974639415741, ファイルパス: ./images/000000188002.jpg
  ID: 455852426864624282, 距離: 0.775324821472168, ファイルパス: ./images/000000544238.jpg
  ID: 455852426864624218, 距離: 0.747927188873291, ファイルパス: ./images/000000544240.jpg
  ID: 455852426864624312, 距離: 0.7380860447883606, ファイルパス: ./images/000000544325.jpg
